In [3]:
import time, json
from pathlib import Path
import requests
import pandas as pd

In [7]:
TESTS = [
    # Hindi
    ("hi-IN", "hi", "native", "general", "आज मौसम साफ है और हम खेत में काम कर रहे हैं।"),
    ("hi-IN", "hi", "native", "agri", "गेहूं की फसल में पीले धब्बे दिख रहे हैं, क्या दवा डालनी चाहिए?"),
    ("hi-IN", "hi", "native", "numbers", "आज की तारीख 27/02/2026 है और मेरा मोबाइल नंबर 9876543210 है।"),
    ("hi-IN", "hi", "roman",  "roman_numbers", "Aaj ki tareekh 27/02/2026 hai aur mera mobile number 9876543210 hai."),

    # Punjabi
    ("pa-IN", "pa", "native", "general", "ਅੱਜ ਮੌਸਮ ਚੰਗਾ ਹੈ ਅਤੇ ਅਸੀਂ ਖੇਤ ਵਿੱਚ ਕੰਮ ਕਰ ਰਹੇ ਹਾਂ।"),
    ("pa-IN", "pa", "native", "numbers", "ਅੱਜ ਦੀ ਤਾਰੀਖ 27/02/2026 ਹੈ ਤੇ ਮੇਰਾ ਮੋਬਾਈਲ ਨੰਬਰ 9876543210 ਹੈ।"),
    ("pa-IN", "pa", "roman",  "roman_numbers", "Ajj di tareekh 27/02/2026 ae te mera mobile number 9876543210 ae."),

    # Bengali
    ("bn-IN", "bn", "native", "general", "আজ আবহাওয়া ভালো এবং আমরা মাঠে কাজ করছি।"),
    ("bn-IN", "bn", "native", "numbers", "আজকের তারিখ 27/02/2026 এবং আমার মোবাইল নম্বর 9876543210।"),
    ("bn-IN", "bn", "roman",  "roman_numbers", "Aajker tarikh 27/02/2026 ebong amar mobile number 9876543210."),

    # Gujarati
    ("gu-IN", "gu", "native", "general", "આજે હવામાન સારું છે અને અમે ખેતરમાં કામ કરી રહ્યા છીએ।"),
    ("gu-IN", "gu", "native", "numbers", "આજની તારીખ 27/02/2026 છે અને મારો મોબાઇલ નંબર 9876543210 છે।"),
    ("gu-IN", "gu", "roman",  "roman_numbers", "Aajni tarik 27/02/2026 chhe ane maro mobile number 9876543210 chhe."),

    # Tamil
    ("ta-IN", "ta", "native", "general", "இன்று வானிலை நல்லதாக உள்ளது, நாங்கள் வயலில் வேலை செய்கிறோம்."),
    ("ta-IN", "ta", "native", "numbers", "இன்றைய தேதி 27/02/2026 மற்றும் என் கைபேசி எண் 9876543210."),
    ("ta-IN", "ta", "roman",  "roman_numbers", "Inraiya thethi 27/02/2026; en mobile number 9876543210."),

    # Mixed / Hinglish (force Hindi voice for fairness)
    ("hi-IN", "mix", "mixed", "hinglish", "Kal mandi rate check karna hai, please update kar dena."),
]

In [4]:
XI_API_KEY = "PLACEHOLDER"

BASE = "https://api.elevenlabs.io/v1"
OUT_DIR = Path("elevenlabs_outputs")
OUT_DIR.mkdir(exist_ok=True)

headers = {
    "xi-api-key": XI_API_KEY,
}

In [5]:
r = requests.get(f"{BASE}/voices", headers=headers, timeout=30)
print("status:", r.status_code)
data = r.json()
voices = data.get("voices", [])
print("voices found:", len(voices))

for i, v in enumerate(voices[:10]):
    print(i, "|", v["name"], "|", v["voice_id"])

VOICE_ID = voices[0]["voice_id"] if voices else None
print("Using VOICE_ID:", VOICE_ID)

status: 200
voices found: 21
0 | Roger - Laid-Back, Casual, Resonant | CwhRBWXzGAHq8TQ4Fs17
1 | Sarah - Mature, Reassuring, Confident | EXAVITQu4vr4xnSDxMaL
2 | Laura - Enthusiast, Quirky Attitude | FGY2WhTYpPnrIDTdsKH5
3 | Charlie - Deep, Confident, Energetic | IKne3meq5aSn9XLyUdCD
4 | George - Warm, Captivating Storyteller | JBFqnCBsd6RMkjVDRZzb
5 | Callum - Husky Trickster | N2lVS1w4EtoT3dr4eOWO
6 | River - Relaxed, Neutral, Informative | SAz9YHcvj6GT2YYXdXww
7 | Harry - Fierce Warrior | SOYHLrjzK2X1ezoPC6cr
8 | Liam - Energetic, Social Media Creator | TX3LPaxmHKxFdv7VOQHJ
9 | Alice - Clear, Engaging Educator | Xb7hH8MSUJpSbSDYk0k2
Using VOICE_ID: CwhRBWXzGAHq8TQ4Fs17


In [11]:
MODEL_ID = "eleven_v3"
OUTPUT_FORMAT = "wav"

def elevenlabs_tts_to_wav(text: str, out_path: Path):
    url = f"{BASE}/text-to-speech/{VOICE_ID}"

    payload = {
        "text": text,
        "model_id": MODEL_ID,
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 0.75
        }
    }

    t0 = time.time()
    r = requests.post(
        url,
        headers={**headers, "Content-Type": "application/json", "Accept": "audio/wav"},
        json=payload,
        timeout=90
    )
    latency = time.time() - t0

    if r.status_code != 200:
        return {"ok": False, "status": r.status_code, "latency": latency, "error": r.text[:500]}

    out_path.write_bytes(r.content)
    return {"ok": True, "status": 200, "latency": latency, "bytes": len(r.content)}

In [12]:
MODEL_NAME = "elevenlabs_v3"
rows = []

for locale, lang, script, tid, text in TESTS:
    fn = f"{MODEL_NAME}_{lang}_{script}_{tid}.wav"
    out_path = OUT_DIR / fn

    res = elevenlabs_tts_to_wav(text, out_path)

    rows.append({
        "input_locale": locale,
        "lang": lang,
        "script": script,
        "test_id": tid,
        "model_id": MODEL_ID,
        "voice_id": VOICE_ID,
        "ok": res.get("ok", False),
        "status": res.get("status"),
        "latency_s": round(res.get("latency", -1), 3),
        "bytes": res.get("bytes", 0),
        "error": res.get("error", ""),
        "file": str(out_path),
        "text_preview": text[:80],
    })

    print(lang, script, tid, "->", "OK" if res.get("ok") else f"FAIL ({res.get('status')})")

df = pd.DataFrame(rows)
df.to_csv(OUT_DIR / "results.csv", index=False)
df

hi native general -> OK
hi native agri -> OK
hi native numbers -> OK
hi roman roman_numbers -> OK
pa native general -> OK
pa native numbers -> OK
pa roman roman_numbers -> OK
bn native general -> OK
bn native numbers -> OK
bn roman roman_numbers -> OK
gu native general -> OK
gu native numbers -> OK
gu roman roman_numbers -> OK
ta native general -> OK
ta native numbers -> OK
ta roman roman_numbers -> OK
mix mixed hinglish -> OK


,input_locale,lang,script,test_id,model_id,voice_id,ok,status,latency_s,bytes,error,file,text_preview
0,hi-IN,hi,native,general,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,1.478,53542,,elevenlabs_outputs/elevenlabs_v3_hi_native_gen...,आज मौसम साफ है और हम खेत में काम कर रहे हैं।
1,hi-IN,hi,native,agri,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,2.079,62319,,elevenlabs_outputs/elevenlabs_v3_hi_native_agr...,"गेहूं की फसल में पीले धब्बे दिख रहे हैं, क्या ..."
2,hi-IN,hi,native,numbers,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,2.560,140478,,elevenlabs_outputs/elevenlabs_v3_hi_native_num...,आज की तारीख 27/02/2026 है और मेरा मोबाइल नंबर ...
3,hi-IN,hi,roman,roman_numbers,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,2.533,158450,,elevenlabs_outputs/elevenlabs_v3_hi_roman_roma...,Aaj ki tareekh 27/02/2026 hai aur mera mobile ...
4,pa-IN,pa,native,general,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,2.592,63991,,elevenlabs_outputs/elevenlabs_v3_pa_native_gen...,ਅੱਜ ਮੌਸਮ ਚੰਗਾ ਹੈ ਅਤੇ ਅਸੀਂ ਖੇਤ ਵਿੱਚ ਕੰਮ ਕਰ ਰਹੇ ...
5,pa-IN,pa,native,numbers,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,3.259,139224,,elevenlabs_outputs/elevenlabs_v3_pa_native_num...,ਅੱਜ ਦੀ ਤਾਰੀਖ 27/02/2026 ਹੈ ਤੇ ਮੇਰਾ ਮੋਬਾਈਲ ਨੰਬਰ...
6,pa-IN,pa,roman,roman_numbers,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,2.621,163466,,elevenlabs_outputs/elevenlabs_v3_pa_roman_roma...,Ajj di tareekh 27/02/2026 ae te mera mobile nu...
7,bn-IN,bn,native,general,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,1.785,43511,,elevenlabs_outputs/elevenlabs_v3_bn_native_gen...,আজ আবহাওয়া ভালো এবং আমরা মাঠে কাজ করছি।
8,bn-IN,bn,native,numbers,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,3.341,155942,,elevenlabs_outputs/elevenlabs_v3_bn_native_num...,আজকের তারিখ 27/02/2026 এবং আমার মোবাইল নম্বর 9...
9,bn-IN,bn,roman,roman_numbers,eleven_v3,CwhRBWXzGAHq8TQ4Fs17,True,200,2.660,172661,,elevenlabs_outputs/elevenlabs_v3_bn_roman_roma...,Aajker tarikh 27/02/2026 ebong amar mobile num...
